In [1]:
with open('../stringdumps/stringdump_0_44_09.txt', 'rb') as first:
    first = {line.rstrip(b'\r\n'): i for i, line in enumerate(first.readlines())}

In [2]:
with open('../stringdumps/stringdump_0_44_12.txt', 'rb') as second:
    second = {line.rstrip(b'\r\n'): i for i, line in enumerate(second.readlines())}

In [3]:
diff = sorted(second.keys()-first.keys(), key=lambda s: (len(s), second[s],))

In [4]:
for item in diff:
    print(second[item]+1, item)

26573 b'at '
26709 b'by '
26561 b'upon '
26742 b'that '
27997 b'Expel'
12304 b'Done: '
26556 b'while '
26623 b'being '
26641 b'to be '
26700 b'having'
26792 b'a bath'
26852 b'about '
3539 b'a ghost'
11664 b'on the '
26577 b'seeing '
26725 b'eating '
27977 b': Expel'
33259 b' remain'
11662 b'You last'
12329 b': Expand'
12355 b'unknown.'
14611 b': Legend'
18065 b'expelled'
25533 b'changed '
25552 b'allowed '
26591 b'viewing '
26811 b'writing '
26812 b'reading '
28000 b'Child is'
28002 b'nobility'
28024 b'Nobility'
28026 b'Does the'
7844 b'MESSENGER'
14639 b': Request'
19390 b'Surrender'
21814 b'A_LOG_PIN'
25544 b'reliving '
26579 b'while in '
26691 b'uncovered'
26809 b'learning '
26829 b'teaching '
28004 b'Spouse is'
28023 b'expelling'
32014 b'Expulsion'
411 b', has been'
4859 b'messengers'
12306 b'Guided by '
12360 b': Collapse'
14612 b' messenger'
18890 b' expelled '
21811 b'A_LOG_SORT'
25543 b' [C:5:0:1]'
25546 b' [C:5:0:0]'
26559 b'mastering '
26562 b'improving '
26575 b"somebody's"
